In [1]:

import numpy as np
import scipy as sp
import h5py
import os

import SimulatorData as sim_data
import Simulator as sim_system


In [2]:

##! keys values are the columns of interest of exp data,
##! but the values are the ones used by the simulator 
##* the experimental data is read as it is presented, modfications are added later in the Simaulator object
##* gas species must be consistent with the chemical equations from the json file


schema = {
    "Pressure":     "pressure",
    "CurrentName":  "current",
    "WallTemp":     "Tw",
    "T near wall":  "Tnw", 
    "O mean":       "O",
    "N":            "N",
    "CO mean":      "CO",
    "gamma.1":      "gamma_exp"
}


###* Input files and function

reactions_file = "reactionsSimple.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.6, 'Vdb_S':0.1, 'Odb_S': 0.1}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO2' :  lambda const_dict, exp_dict: compute_remaining_flux(const_dict, exp_dict, 0.032),
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'O', 0.016),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

input_folder_path = "Experimental_data_Paper"

output_folder_path = "Buffer_Data"
exp_data_file = input_folder_path + ".hdf5"

In [3]:

data_loader = sim_data.DataLoader(schema, folder_path=input_folder_path, output_file=exp_data_file, output_folder=output_folder_path)
data_loader.load_data(force_update=True)

Summary Buffer:
File Data : [b'Experimental_data_Paper/gammas_TD2019.xlsx']
col CO: shape(66,)
col File_Data: shape(1,)
col N: shape(66,)
col O: shape(66,)
col Tnw: shape(66,)
col Tw: shape(66,)
col current: shape(66,)
col gamma_exp: shape(66,)
col pressure: shape(66,)


In [4]:


exp_file = os.path.join(output_folder_path, exp_data_file)

sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)


print(sim.output_parser['rates_model'])
print(sim.output_parser['reactions_list'])

### 12, 14 


### 21, 22



dO_S/dt = -1.0*O_F*O_S*r_7 + 1.0*O_F*r_5*(-O_S - Odb_S - Vdb_S + 1.0) - 1.0*O_S*r_16 - 1.0*O_S*r_17 - 1.0*O_S*r_6 + 1.0*r_3*(-O_S - Odb_S - Vdb_S + 1.0)
dOdb_S/dt = -1.0*O_F*Odb_S*r_27 + 1.0*O_F*Vdb_S*r_26 - 1.0*Odb_S*r_23 - 1.0*Odb_S*r_24 - 1.0*Odb_S*r_25 + 1.0*Vdb_S*r_20
dVdb_S/dt = 1.0*O_F*Odb_S*r_27 - 1.0*O_F*Vdb_S*r_26 + 1.0*O_S*r_16 + 1.0*O_S*r_17 + 1.0*Odb_S*r_25 - 1.0*Vdb_S*r_20 - 1.0*Vdb_S*r_21 - 1.0*Vdb_S*r_22 + 1.0*r_18*(-O_S - Odb_S - Vdb_S + 1.0) + 1.0*r_19*(-O_S - Odb_S - Vdb_S + 1.0)
dO2_F/dt = -1.0*O2_F*O_F*r_15 - 1.0*O2_F*r_10 - 1.0*O2_F*r_12 - 1.0*O2_F*r_14 + 1.0*r_9*(-O2_F - O_F + 1.0)
dO_F/dt = -1.0*O2_F*O_F*r_15 - 2.0*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - 1.0*O_F*r_11 - 1.0*O_F*r_2 - 1.0*O_F*r_4 - 0.02*O_F*r_5*(-O_S - Odb_S - Vdb_S + 1.0) + 1.0*r_1*(-O2_F - O_F + 1.0)
r_1 14387.605280283622 |r_2 645547731.6715667 |r_3 14387.605280283622 |r_4 14387.605280283622 |r_5 6025948880.178593 |r_6 14.912525716375674 |r_7 600136.101381866

In [5]:



frac_solutions_arr, gammas_result_arr = sim.solve_system()

gammas_sum = np.array([sum(gamma_dict.values()) for gamma_dict in gammas_result_arr])


print(sim.output_parser['species_model'])
print(frac_solutions_arr[:5])
print(gammas_result_arr[:5])

['O_S', 'Odb_S', 'Vdb_S', 'O2_F', 'O_F']
[[9.76230934e-01 1.17912304e-02 1.17918945e-02 3.33219600e-07
  2.22699283e-05]
 [9.90125793e-01 4.83874643e-03 4.83901403e-03 4.75777165e-07
  3.71978757e-05]
 [9.99085201e-01 3.54322737e-04 3.54346345e-04 6.21435625e-07
  4.58429899e-05]
 [9.99766722e-01 3.50306925e-11 3.50300297e-11 1.08718955e-06
  7.35826524e-05]
 [9.99748750e-01 9.18266568e-11 9.18245081e-11 1.40051736e-06
  8.72532288e-05]]
[{'r_4': 4.366652611050416e-05, 'r_6': 3.9680311779441585e-05, 'r_7': 3.556251296270382e-05, 'r_8': 0.0005430541379423504, 'r_11': 0.00012387847535737383, 'r_12': 1.8535639362188733e-06, 'r_14': 3.266858822119513e-07, 'r_15': 4.06279445480943e-06, 'r_23': 1.3118461315100727e-08, 'r_24': 4.624190214586675e-09, 'r_25': 0.0004623965674906219, 'r_27': 3.4652843086576867e-06}
 {'r_4': 7.293701122366465e-05, 'r_6': 4.344865165119389e-05, 'r_7': 3.605219353835143e-05, 'r_8': 0.0009066588187087879, 'r_11': 0.00017679793456954528, 'r_12': 2.26132322005021e-06, 

In [6]:

print(gammas_sum)


[0.00125796 0.0014356  0.00154653 0.0024537  0.00294202 0.00332242
 0.00154125 0.00168082 0.00172022 0.00218509 0.00282171 0.00332337
 0.00395099 0.00458441 0.00484624 0.00058163 0.00049913 0.00046387
 0.00053143 0.00068255 0.00080385 0.00097211 0.00110273 0.00079454
 0.00061466 0.00052725 0.00061035 0.00077691 0.00091746 0.00110741
 0.0013201  0.00145621 0.00045043 0.00034063 0.00027484 0.00029706
 0.00035499 0.00040116 0.00046218 0.00053802 0.0006347  0.00043166
 0.0003091  0.00032839 0.00039376 0.00045037 0.00054104 0.00067284
 0.00074725 0.00042904 0.00033168 0.0002702  0.00027877 0.00031038
 0.00033698 0.00037796 0.00043303 0.00059027 0.00041135 0.00029744
 0.00030642 0.00034671 0.00038439 0.00045262 0.00056855 0.00066593]


In [7]:
# [0.00079391 0.0012452  0.00153259 0.0024537  0.00294202 0.00332242
#  0.00083535 0.00139788 0.00169989 0.00218368 0.00282171 0.00332337
#  0.00395099 0.00458441 0.00484624 0.00026752 0.00036734 0.00045403
#  0.00053071 0.00068255 0.00080385 0.00097211 0.00110273 0.0002879
#  0.00040623 0.00051199 0.00060926 0.00077691 0.00091746 0.00110741
#  0.0013201  0.00145621 0.00020071 0.0002355  0.00026695 0.00029648
#  0.00035499 0.00040116 0.00046218 0.00053802 0.00021855 0.00025846
#  0.00029629 0.00032747 0.00039376 0.00045037 0.00054104 0.00067284
#  0.00074725 0.00023204 0.00024819 0.00026391 0.00027831 0.00031038
#  0.00033698 0.00037796 0.00043303 0.00024911 0.00026795 0.00028674
#  0.00030564 0.00034671 0.00038439 0.00045262 0.00056855 0.00066593]